<a href="https://colab.research.google.com/github/stellagerantoni/LatentCfMultivariate/blob/main/FingerMovements_evaluate_without_KDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/stellagerantoni/LatentCfMultivariate

Cloning into 'LatentCfMultivariate'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 137 (delta 79), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (137/137), 7.21 MiB | 7.82 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [2]:
!pip install -q wildboar
!pip install -q scikit-learn
!pip install -q stumpy
!pip install -q fastdtw
!pip install aeon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 15.5 MB/s eta 0:00:00


In [3]:
import logging
import os
import warnings
from argparse import ArgumentParser
from aeon.datasets import load_classification

from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.spatial import distance_matrix
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from wildboar.datasets import load_dataset
from wildboar.ensemble import ShapeletForestClassifier
from wildboar.explain.counterfactual import counterfactuals
%cd '/content/LatentCfMultivariate'
from _guided import ModifiedLatentCF
from help_functions import *
from keras_models import *

/content/LatentCfMultivariate


In [4]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
RANDOM_STATE = 39

## **FUNCTIONS**

In [5]:
def load_dataset(dataset):
  X, y = load_classification(dataset)
  if dataset == 'Heartbeat':
    pos = 'normal'
    neg = 'abnormal'
  if dataset == 'SelfRegulationSCP1':
    pos = 'positivity'
    neg = 'negativity'
  if dataset == 'Blink':
    pos = 'shortblink'
    neg = 'longblink'
    X = np.array(X)
    y = np.array(y)
  if dataset == 'FingerMovements':
    pos = 'left'
    neg = 'right'
  if dataset == 'Cricket':
    return X,y,meta_data
  if dataset == 'SpokenArabicDigits':
    return X,y,meta_data
  if dataset == 'PenDigits':
    return X,y,meta_data


  print(" Shape of X = ", X.shape)
  print(" Shape of y = ", y.shape)
  #print(" Meta data = ", meta_data)
  # Convert positive and negative labels to 1 and 0
  pos_label, neg_label = 1, 0
  if pos != pos_label:
      y[y==pos] = pos_label # convert/normalize positive label to 1
  if neg != neg_label:
      y[y==neg] = neg_label # convert negative label to 0

  y = y.astype(int)
  print(f"\n X[:1] = \n{X[:1]}")
  return X,y,pos_label, neg_label

# **ACTUALL CODE**
datasets available : 'Heartbeat', 'SelfRegulationSCP1'

In [6]:
RANDOM_STATE = 39
X,y,pos_label,neg_label = load_dataset('FingerMovements')
X = X.transpose(0,2,1)
print(f'shape of X = {X.shape}')
print(f'shape of y = {y.shape}')
#print(f'data imformation = {data_information}')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
print(f'shape of X train = {X_train.shape}')
print(f'shape of y train = {y_train.shape}')

 Shape of X =  (416, 28, 50)
 Shape of y =  (416,)

 X[:1] = 
[[[41.8 44.8 47.1 ... 69.8 72.6 76.1]
  [55.2 53.8 59.9 ... 17.5 28.  12.1]
  [-8.6 -3.6 14.4 ... 23.3 35.9 23.2]
  ...
  [16.9 24.5 24.5 ... 51.9 59.6 57.3]
  [42.2 35.  41.7 ... 51.5 58.5 46.9]
  [13.  26.6 52.5 ... -3.5 -3.2 -2.6]]]
shape of X = (416, 50, 28)
shape of y = (416,)
shape of X train = (332, 50, 28)
shape of y train = (332,)


In [7]:
# Upsample the minority class
unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f'before: {class_counts}')
X_train,y_train = upsample_minority_multivariate(X_train,y_train)
unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f'after: {class_counts}')

before: [166 166]
after: [166 166]


In [8]:
n_training,n_timesteps, n_features= X_train.shape

X, trained_scaler =  normalize_multivariate(data=X, n_timesteps=n_timesteps, n_features = n_features)
X_train_processed, trained_scaler =  normalize_multivariate(data=X_train, n_timesteps=n_timesteps, n_features = n_features)
X_test_processed, _ =  normalize_multivariate(data=X_test, n_timesteps=n_timesteps, scaler=trained_scaler, n_features = n_features)

X, padding_size = conditional_pad_multivariate(X)
X_train_processed_padded, padding_size = conditional_pad_multivariate(X_train_processed) # add extra padding zeros if n_timesteps cannot be divided by 4, required for 1dCNN autoencoder structure
X_test_processed_padded, _ = conditional_pad_multivariate(X_test_processed)

n_timesteps_padded = X_train_processed_padded.shape[1]
print(f"Data pre-processed, original #timesteps={n_timesteps}, padded #timesteps={n_timesteps_padded}.")

#check the processing (0,1) min should be min 0 and max should be max 1
print(f"\nmin value = {np.min(X_train)}, max value = {np.max(X_train)}")
print(f"min value normalized = {np.min(X_train_processed)}, max value normalized= {np.max(X_train_processed)}")

#check that padding paddes the right dimention
print(f"\nX_train.shape = {X_train.shape}" )
print(f"X_train_processed_padded.shape = {X_train_processed_padded.shape}")


Data pre-processed, original #timesteps=50, padded #timesteps=52.

min value = -132.1, max value = 205.1
min value normalized = 0.0, max value normalized= 1.0

X_train.shape = (332, 50, 28)
X_train_processed_padded.shape = (332, 52, 28)


In [9]:
from sklearn.model_selection import train_test_split
X_train_processed_padded,X_validation, y_train, y_validation = train_test_split(X_train_processed_padded, y_train, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train)


In [10]:
print(f'X_train = {X_train_processed_padded.shape}')
print(f'X_validation = {X_validation.shape}')
print(f'X_test = {X_test_processed_padded.shape}')

y_classes = y
y_train_classes = y_train
y_validation_classes = y_validation
y_test_classes = y_test

from tensorflow.keras.utils import to_categorical
y = to_categorical(y,len(np.unique(y)))
y_train = to_categorical(y_train, len(np.unique(y_train)))
y_validation = to_categorical(y_validation, len(np.unique(y_validation)))
y_test = to_categorical(y_test, len(np.unique(y_test)))

print(f'\ny_train_classes = {y_train_classes.shape}, y_validation_classes = {y_validation_classes.shape}, y_test_classes = {y_test_classes.shape}')
print(f'y_train = {y_train.shape}, y_validation = {y_validation.shape}, y_test= {y_test.shape}')

X_train = (265, 52, 28)
X_validation = (67, 52, 28)
X_test = (84, 52, 28)

y_train_classes = (265,), y_validation_classes = (67,), y_test_classes = (84,)
y_train = (265, 2), y_validation = (67, 2), y_test= (84, 2)


In [11]:


# ## 2. LatentCF models
# reset seeds for numpy, tensorflow, python random package and python environment seed
reset_seeds()
###############################################
# ### 1dCNN classifier

cnnClassifier = Classifier(
    n_timesteps_padded, n_features, n_output=2, add_dense_layer = False
)

optimizer = keras.optimizers.Adam(lr=0.001)
cnnClassifier.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
)

# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=15, restore_best_weights=True
)
# Train the model
reset_seeds()
print("Training log for LSTM-FCN classifier:")
classifier_history = cnnClassifier.fit(
    X_train_processed_padded,
    y_train,
    epochs=150,
    batch_size=12,
    shuffle=True,
    verbose=True,
    validation_data=(X_validation, y_validation),
    callbacks=[early_stopping_accuracy],
)

y_pred = cnnClassifier.predict(X_test_processed_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
acc = balanced_accuracy_score(y_true=y_test_classes, y_pred=y_pred_classes)
print(f"LSTM-FCN classifier trained, with validation accuracy {acc}.")

confusion_matrix_df = pd.DataFrame(
    confusion_matrix(y_true=y_test_classes, y_pred=y_pred_classes, labels=[1, 0]),
    index=["True:8", "True:0"],
    columns=["Pred:8", "Pred:0"],
)
print(confusion_matrix_df)


Training log for LSTM-FCN classifier:
Epoch 1/150
23/23 [==============================] - 1s 16ms/step - loss: 0.8307 - accuracy: 0.5132 - val_loss: 0.7577 - val_accuracy: 0.4925
Epoch 2/150
23/23 [==============================] - 0s 7ms/step - loss: 0.6913 - accuracy: 0.6226 - val_loss: 0.7282 - val_accuracy: 0.5224
Epoch 3/150
23/23 [==============================] - 0s 7ms/step - loss: 0.6332 - accuracy: 0.7132 - val_loss: 0.7123 - val_accuracy: 0.6567
Epoch 4/150
23/23 [==============================] - 0s 8ms/step - loss: 0.7008 - accuracy: 0.6189 - val_loss: 0.7063 - val_accuracy: 0.6418
Epoch 5/150
23/23 [==============================] - 0s 7ms/step - loss: 0.5797 - accuracy: 0.7585 - val_loss: 0.7009 - val_accuracy: 0.6567
Epoch 6/150
23/23 [==============================] - 0s 7ms/step - loss: 0.5776 - accuracy: 0.7736 - val_loss: 0.7315 - val_accuracy: 0.4925
Epoch 7/150
23/23 [==============================] - 0s 7ms/step - loss: 0.5878 - accuracy: 0.7321 - val_loss: 0.73

In [12]:
reset_seeds()


# ### 1dCNN autoencoder
autoencoder = Autoencoder(n_timesteps_padded,n_features,32)
optimizer = keras.optimizers.Adam(lr=0.0005)
autoencoder.compile(optimizer=optimizer, loss="mse")

# Define the early stopping criteria
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, restore_best_weights=True)
# Train the model
reset_seeds()
print("Training log for 1dCNN autoencoder:")
autoencoder_history = autoencoder.fit(
    X_train_processed_padded,
    X_train_processed_padded,
    epochs=50,
    batch_size=12,
    shuffle=True,
    verbose=2,
    validation_data=(X_validation, X_validation),
    callbacks=[early_stopping])

ae_val_loss = np.min(autoencoder_history.history['val_loss'])
print(f"1dCNN autoencoder trained, with validation loss: {ae_val_loss}.")


(None, 52, 28)
(None, 52, 32)
(None, 26, 32)
(None, 26, 16)
(None, 13, 16)
(None, 13, 16)
(None, 26, 16)
(None, 26, 32)
(None, 52, 32)
(None, 52, 28)
Training log for 1dCNN autoencoder:
Epoch 1/50
23/23 - 2s - loss: 0.1518 - val_loss: 0.0500 - 2s/epoch - 68ms/step
Epoch 2/50
23/23 - 0s - loss: 0.0250 - val_loss: 0.0140 - 126ms/epoch - 5ms/step
Epoch 3/50
23/23 - 0s - loss: 0.0131 - val_loss: 0.0112 - 117ms/epoch - 5ms/step
Epoch 4/50
23/23 - 0s - loss: 0.0112 - val_loss: 0.0099 - 130ms/epoch - 6ms/step
Epoch 5/50
23/23 - 0s - loss: 0.0101 - val_loss: 0.0092 - 139ms/epoch - 6ms/step
Epoch 6/50
23/23 - 0s - loss: 0.0094 - val_loss: 0.0083 - 125ms/epoch - 5ms/step
Epoch 7/50
23/23 - 0s - loss: 0.0087 - val_loss: 0.0077 - 130ms/epoch - 6ms/step
Epoch 8/50
23/23 - 0s - loss: 0.0080 - val_loss: 0.0071 - 114ms/epoch - 5ms/step
Epoch 9/50
23/23 - 0s - loss: 0.0074 - val_loss: 0.0066 - 118ms/epoch - 5ms/step
Epoch 10/50
23/23 - 0s - loss: 0.0068 - val_loss: 0.0060 - 125ms/epoch - 5ms/step
Epoch

In [13]:
from _guided import get_global_weights
PRED_MARGIN_W_LIST = [1]
from help_functions import evaluate
w_type = "global"
pos_label = 1
neg_label = 0

if w_type == "global":
    step_weights = get_global_weights(
        X,
        y_classes,
        cnnClassifier,
        n_timesteps= n_timesteps,
        n_features=n_features,
        random_state=RANDOM_STATE,
    )
elif w_type == "uniform":
    step_weights = np.ones((1, n_timesteps_padded, n_features))
elif w_type.lower() == "local":
    step_weights = "local"
else:
    raise NotImplementedError(
        "A.w_type not implemented, please choose 'local', 'global' or 'uniform'."
    )
### Evaluation metrics
for pred_margin_weight in PRED_MARGIN_W_LIST:
    print(f"The current prediction margin weight is {pred_margin_weight}.")

    lr_list = [0.001]


13/13 [==============================] - 0s 2ms/step
The current prediction margin weight is 1.


In [56]:
reset_seeds()
cf_model = ModifiedLatentCF(
    probability=0.5,tolerance=1e-6, max_iter=500, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),autoencoder = autoencoder,
    pred_margin_weight=0.7, step_weights = step_weights, random_state= RANDOM_STATE)
cf_model.fit(cnnClassifier)

y_neg = y_classes[y_classes == 0]
X_neg = X[y_classes == 0]


with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning) # ignore warning of matrix multiplication: https://stackoverflow.com/questions/29688168/mean-nanmean-and-warning-mean-of-empty-slice
    cf_embeddings, losses, weights = cf_model.transform(X_neg, y_neg) #self, x, pred_label
cf_pred_labels = cnnClassifier.predict(cf_embeddings)[:,1]# predicted probabilities of CFs
for idx in range(cf_pred_labels.shape[0]):
  if cf_pred_labels[idx] > 0.5:
    cf_pred_labels[idx] = 1
  else:
    cf_pred_labels[idx] = 0

print(f'Transformation_finished with validity_score = {validity_score(y_neg,cf_pred_labels)}')

1 samples been transformed.


ValueError: Input 0 of layer "model_5" is incompatible with the layer: expected shape=(None, 52, 28), found shape=(1, 50, 28)

In [ ]:
#Calculating proximity
from tensorflow.keras.losses import MeanSquaredError
total = 0
probability = 0.5
for idx in range(cf_embeddings.shape[0]):
    counterfactual = cf_embeddings[idx,np.newaxis]
    prediction = cnnClassifier.predict(counterfactual)[:, 1]
    dist = (prediction - probability)
    total +=dist
mean_mse = total /cf_embeddings.shape[0]
print(f"The Mean MSE of the data is: {mean_mse} ")

In [ ]:
#Calculating proximity
from tensorflow.keras.losses import MeanSquaredError
total = 0
probability = 0.5
for idx in range(cf_embeddings.shape[0]):
    counterfactual = cf_embeddings[idx,np.newaxis]
    prediction = cnnClassifier.predict(counterfactual)[:, 1]
    dist = abs(prediction - probability)
    total +=dist
mean_mse = total /cf_embeddings.shape[0]
print(f"The Absolute Mean MSE of the data is: {mean_mse} ")

In [17]:
def remove_paddings(cf_samples, padding_size):
    if padding_size != 0:
        # use np.squeeze() to cut the last time-series dimension, for evaluation
        cf_samples = np.squeeze(cf_samples[:, :-padding_size, :])
    else:
        cf_samples = np.squeeze(cf_samples)
    return cf_samples

In [19]:
X = remove_paddings(X, padding_size)
cf_embeddings = remove_paddings(cf_embeddings, padding_size)


In [21]:
#Proximity
X_neg = X[y_classes == 0][10:15]
def euclidean_distance(X, cf_samples):
    paired_distances = np.linalg.norm(X - cf_samples, axis=1)
    return np.mean(paired_distances)
euclidean_distance(X_neg, cf_embeddings)

0.4080537770740321

In [ ]:
for i in range(cf_embeddings.shape[2]):
  print(i)

In [30]:
X[y_classes == 1].shape

(208, 50, 28)

In [47]:
diffrences_from_abnormal = []
diffrences_from_normal = []
for dimention in range(cf_embeddings.shape[2]):


  abnormal_data = X[y_classes == 1][:,:,dimention]
  normal_data = X[y_classes == 0][:,:,dimention]
  counterf_data = cf_embeddings[:,:,dimention]

  #get the kernel for every dimention of the trained
  kernel = gaussian_kde(abnormal_data.T,bw_method=None)

  #get all the log likelihoods
  log_likelihood_abnormal = np.mean(kernel.logpdf(abnormal_data.T))
  log_likelihood_normal = np.mean(kernel.logpdf(normal_data.T))
  log_likelihood_counterfactual = np.mean(kernel.logpdf(counterf_data.T))

  #get the diffrences from the counterfactuals
  diff_from_abnormal = abs(log_likelihood_counterfactual-log_likelihood_abnormal)
  diffrences_from_abnormal.append(diff_from_abnormal)

  diff_from_normal = abs(log_likelihood_counterfactual-log_likelihood_normal)
  diffrences_from_normal.append(diff_from_normal)



In [48]:
print(diffrences_from_normal)

[8.39591427432876, 22.561603009647428, 11.555447980262628, 11.40777506674003, 16.727141753095367, 9.793046527887725, 3.545005317619058, 3.5589713440128747, 10.67552248646274, 19.818223035917654, 11.388448228547333, 14.505243787799103, 16.71963813293864, 15.043047670925105, 19.679160112768898, 10.623244582846098, 13.67547866795762, 9.527591736931385, 13.574705872443033, 19.978713351046423, 4.445560623348996, 8.451266043714355, 7.464364744605788, 14.134139869175314, 10.271067273074564, 17.79688147728423, 18.144336629880428, 12.781409640847926]


In [49]:
print(diffrences_from_abnormal)

[32.43845091138091, 63.671702585101755, 53.418240227547486, 53.93769183473394, 26.21923397531451, 34.057437159706424, 47.56351749557041, 38.9966390628212, 29.539481912882763, 20.799848229411623, 29.595662448518553, 26.235679754004124, 24.037778723812252, 55.71046282531164, 61.54305848281177, 32.147889531635656, 29.091505798787125, 35.123376720700804, 58.64474213897202, 64.03918228285416, 38.05274272592641, 31.40559861079916, 32.823171361556675, 24.256004077806452, 28.81315255416807, 22.106548689486573, 22.359336740589754, 28.807975377490976]


In [52]:
print(np.mean(diffrences_from_normal))

12.72296247293248


In [53]:
print(np.mean(diffrences_from_abnormal))

37.337004008560825


In [54]:
#Calculating proximity
from tensorflow.keras.losses import MeanSquaredError
total = 0
probability = 0.5
for idx in range(cf_embeddings.shape[0]):
    counterfactual = cf_embeddings[idx,np.newaxis]
    prediction = cnnClassifier.predict(counterfactual)[:, 1]
    dist = (prediction - probability)
    total +=dist
mean_mse = total /cf_embeddings.shape[0]
print(f"The Mean MSE of the data is: {mean_mse} ")

InvalidArgumentError: Graph execution error:

Detected at node model/dense/BiasAdd defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.1